# Modul1: Einführung in PySpark

## 1.1. Einführung in Apache Spark
Apache Spark ist ein Framework für **verteilte Datenverarbeitung**, das auf großen Clustern effizient arbeitet.

Kernmerkmale:

- **In-Memory-Verarbeitung:** Spark kann Daten im RAM halten, was die Verarbeitung im Vergleich zu diskbasierten Systemen stark beschleunigt.
- **Skalierbarkeit:** Spark läuft auf einem einzelnen Rechner, aber auch auf Hunderten von Nodes in einem Cluster.
- **Vielseitigkeit:** Spark unterstützt Batch-Processing, Streaming, SQL-Abfragen, Machine Learning (MLlib) und Graphverarbeitung (GraphX).




### 1.1.1. Vorteile von Spark gegenüber klassischen Ansätzen

| Vorteil                     | Beschreibung |
|-------------------------------|-------------|
| Geschwindigkeit               | Bis zu 100x schneller als klassische MapReduce-Ansätze, dank In-Memory-Verarbeitung |
| Skalierbarkeit                 | Verarbeitung von Terabytes oder Petabytes an Daten über Cluster hinweg |
| Flexibilität                   | Unterstützt unterschiedliche Datenformate (CSV, JSON, Parquet, Delta) und APIs (Python, Scala, Java, R) |
| Einheitliche Plattform         | Ein Framework für ETL, Analytics, Machine Learning und Streaming |


### 1.1.2. Cluster-Architektur von Spark

Spark arbeitet verteilt auf einem Cluster. Die wichtigsten Komponenten:

- **Cluster Manager**:
  - Zuteilung von Ressourcen im Cluster (z. B. Standalone, YARN, Databricks-eigener Manager)
- **Driver**: 
  - Koordiniert die Berechnungen
  - Plant Tasks und verwaltet die SparkContext / SparkSession
- **Worker Nodes**:
  - Maschinen im Cluster, auf denen Executor-Prozesse laufen
- **Executors**:
  - Führen die Tasks auf den Worker Nodes aus
  - Jeder Executor verwaltet einen Teil des Arbeitsspeichers und der CPU-Kerne


### 1.1.3. Lazy Evaluation

- Spark führt **Transformationen** (z. B. `filter`, `select`) **nicht sofort** aus.
- Erst eine **Action** (z. B. `count()`, `show()`) löst die Berechnung aus.
- Vorteil: Optimierung durch Spark (z. B. Catalyst Optimizer, Pipeline-Fusion).

Beispiel:  

```python
df_filtered = df.filter(df.Age > 30).select("Name")
# Keine Berechnung bisher
df_filtered.show()  # Jetzt werden die Daten berechnet


## 1.2. PySpark im Überblick

PySpark ist die **Python-Schnittstelle** zu Apache Spark.  
Damit können Python-Entwickler Spark-Funktionen nutzen, ohne Scala oder Java lernen zu müssen.

**Kernideen von PySpark:**
- Zugriff auf Spark Core, Spark SQL, MLlib, GraphX und Streaming über Python
- Verteilte Datenverarbeitung auf Clustern
- Lazy Evaluation: Transformationen werden erst ausgeführt, wenn eine Action aufgerufen wird


## 1.3. Komponenten von Apache Spark

| Komponente | Beschreibung |
|------------|-------------|
| Spark Core | Kern von Spark, verantwortlich für Task Scheduling, Speicherverwaltung und Cluster-Management |
| Spark SQL  | Datenanalyse mit SQL-ähnlichen Abfragen, unterstützt DataFrames und Datasets |
| MLlib      | Machine Learning Bibliothek für verteilte ML-Algorithmen |
| Spark Streaming | Verarbeitung von Echtzeit-Datenströmen |
| GraphX     | Graph-Processing API |

PySpark gibt uns Zugang zu all diesen Komponenten direkt aus Python heraus.

## 1.4. DataFrames und Transformationen

In PySpark arbeiten wir meist mit **DataFrames**, die verteilt auf dem Cluster liegen.

Wichtige Konzepte:

- **Transformationen**: z. B. `select()`, `filter()`, `groupBy()` – werden **lazy** ausgeführt
- **Actions**: z. B. `show()`, `count()`, `collect()` – lösen die tatsächliche Berechnung aus


In [0]:
# Beispiel

# 1. SparkSession erstellen

#INFO: Databricks erstellt eine spark Session automatisch, manuelle Erstellung in der Praxis nur notwendig fuer spezielle Settings wie hier am Beispiel gezeigt
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Einführung Apache Spark") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .config("spark.sql.shuffle.partitions", "50") \
    .getOrCreate()

# 2. Ein DataFrame erstellen
data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
columns = ["Name", "Alter"]

df = spark.createDataFrame(data, columns)
df.show()

# 3. Transformation (Lazy)
df_filtered = df.filter(df.Alter > 30).select("Name")

# 4. Action
df_filtered.show()


## 1.5. Narrow vs. Wide Transformations

Spark-Transformationen lassen sich in zwei Typen einteilen:

#### **Narrow Transformations**
- Jede Partition des Input-RDDs/DFs wird nur auf **eine Partition des Outputs** gemappt.
- Kein Daten-Shuffling zwischen Nodes.
- Beispiele: `map()`, `filter()`, `select()`
- Vorteil: Sehr effizient, da keine Daten über das Netzwerk bewegt werden müssen.

#### **Wide Transformations**
- Eine Partition des Outputs hängt von **mehreren Partitionen des Inputs** ab.
- Spark muss Daten zwischen Nodes verschieben → **Shuffle**.
- Beispiele: `groupByKey()`, `reduceByKey()`, `join()`
- Vorteil: notwendig für Aggregationen und komplexe Operationen, aber teurer in Performance.

#### Shuffle in Spark

**Shuffle** = das Umverteilen von Daten über Nodes im Cluster.  
- Entsteht bei **Wide Transformations**
- Spark schreibt Partitionen auf Disk oder Netzwerk, sendet sie an andere Executor-Nodes
- Teuer: Netzwerktransfer + Disk I/O + Sortierung

**Warum ist Shuffle wichtig zu verstehen?**
- Vermeidet Überraschungen bei Performanceproblemen
- Hilft bei der Optimierung von Jobs

In [0]:
# Praxisbeispiel: Narrow vs. Wide Transformationen

data = [("Alice", "Math", 85),
        ("Bob", "Math", 90),
        ("Alice", "Physics", 95),
        ("Bob", "Physics", 80)]
columns = ["Name", "Fach", "Punkte"]

df = spark.createDataFrame(data, columns)

# Narrow Transformation (filter)
df_filtered = df.filter(df.Punkte > 85)
df_filtered.show()

# Wide Transformation (groupBy)
df_grouped = df.groupBy("Name").sum("Punkte")
df_grouped.show()


## 1.6. DataFrames vs. Spark SQL

In Spark gibt es zwei Hauptmethoden, Daten zu verarbeiten: 
* **DataFrames** (API-basiert) und 
* **Spark SQL** (SQL-ähnliche Abfragen)

Obwohl beide auf der gleichen Engine laufen und ähnliche Optimierungen nutzen, unterscheiden sie sich in Arbeitsweise, Syntax und Flexibilität.


### 1.6.1. DataFrames

In [0]:
#**API-basiert**: Python-, Scala-, oder R-Methoden  
#Unterstützt Transformationen und Aktionen direkt im Code  
#Vorteil: Integration mit komplexer Logik, Bedingungen, User-Defined Functions (UDFs)  

#Beispiel:
# DataFrame erstellen
data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
columns = ["Name", "Alter"]

df = spark.createDataFrame(data, columns)

# Transformation: Filter + Select
df_filtered = df.filter(df.Alter > 30).select("Name")
df_filtered.show()


### 1.6.2. SQL

- **SQL-basiert**: Daten mit bekannten SQL-Statements abfragen  
- Vorteil: Leicht verständlich für Analysten, Data Engineers und SQL-Profis  
- Transformationen werden ebenfalls **lazy** geplant und optimiert durch den **Catalyst Optimizer**  

In [0]:
# Beispiel:

# DataFrame als temporäre View registrieren
df.createOrReplaceTempView("personen")

In [0]:
%sql
select * from personen

In [0]:
new_df = spark.sql("select * from personen")
new_df.show()

new_df.write.format("delta").mode("overwrite").saveAsTable("personen_tabelle")

spark.sql("CREATE SCHEMA IF NOT EXISTS neues_schema")

new_df.write.format("delta").mode("overwrite").saveAsTable("neues_schema.personen_tabelle")

#spark.sql("DROP SCHEMA neues_schema CASCADE")
